# **Homework 2**
- ### **Course:** Information Security
- ### **Autores:** Anahí Andrade & Mateo Salgado  
- ### **Fecha:** 13/11/2025  

## **1. Standard Digital Signature Protocol Review**

---

The standard, cryptographically robust protocol for creating a digital signature uses asymmetric cryptography combined with secure hashing to provide assurance of both origin authentication and integrity. The process follows four steps:

- **Digest Computation**: A cryptographic hash function computes a short digest $H(M)$ of the original message $M$.

- **Signing**: The sender encrypts the hash value $H(M)$ using their own Private Key $(SK_{Signer})$. This encrypted hash is the signature $(Sig)$.

- **Transmission**: The sender transmits the original message and the signature: $M, Sig$.

- **Verification**: The receiver decrypts $Sig$ using the sender's publicly available Public Key $(PK_{Signer})$ to recover the original hash value $H(M)$. The receiver then independently computes a hash $H(M')$ on the received message $M'$ and verifies that $H(M') = H(M)$. If they match, integrity and authentication are confirmed.

### **Critique of Flawed Alternatives**

Students must analyze two proposed alternative signature schemes that deviate from the standard protocol and detail which desirable properties are violated.

### **Mechanism 1**:

This scheme proposes using a symmetric key scheme instead of Public Key Cryptography (PKC). To verify the signature, the recipient must give the signature back to the signer, who decrypts and checks it.

### **Mechanism 2**:

The signer encrypts the entire object M with their private key, and the reader decrypts the entire resulting ciphertext using the signer's public key to both read and verify the content.

## **1. Solución**

---

### **Mechanism 1:**

Este mecanismo propone usar criptografía simétrica en lugar de criptografía de clave pública. En el caso de **no repudio**, esta propiedad es **violada** porque tanto el firmante como el verificador comparten la misma clave secreta, lo que significa que ambas partes pueden generar firmas válidas. Esto hace imposible demostrar ante un tercero quién realmente creó la firma, ya que el verificador podría haber fabricado la firma él mismo y el firmante puede negar haberla creado. En el caso de **autenticación**, esta propiedad es **parcialmente comprometida** porque aunque el verificador puede confirmar que la firma proviene de alguien con la clave compartida, no puede probar ante terceros la identidad del firmante. En el caso de **integridad**, esta propiedad se **mantiene** ya que cualquier modificación al mensaje sería detectada cuando el verificador descifra y verifica la firma. En el caso de **confidencialidad**, esta propiedad **no aplica** directamente a las firmas digitales, aunque el cifrado simétrico podría proporcionar confidencialidad si se implementara adecuadamente, pero ese no es el propósito de una firma digital. En conclusión, la propiedad crítica violada en este mecanismo es el **no repudio**, lo que hace que este esquema sea fundamentalmente inadecuado para un sistema de firmas digitales, ya que no se puede probar de manera irrefutable quién generó la firma.

---

### **Mechanism 2:**

Este mecanismo cifra el mensaje completo con la clave privada del firmante en lugar de firmar solamente el hash del mensaje. En el caso de **no repudio**, esta propiedad se **mantiene** porque solo el poseedor de la clave privada puede cifrar el mensaje, y cualquiera con la clave pública puede verificar que proviene del firmante. En el caso de **autenticación**, esta propiedad se **mantiene** ya que el descifrado exitoso con la clave pública del firmante confirma su identidad. En el caso de **integridad**, esta propiedad se **mantiene parcialmente** porque cualquier modificación haría que el descifrado falle o produzca datos sin sentido, pero al no usar una función hash criptográfica, no hay un mecanismo estandarizado y robusto para detectar modificaciones sutiles. En el caso de **confidencialidad**, esta propiedad es **violada** porque cifrar con la clave privada significa que cualquiera con acceso a la clave pública (que es pública por definición) puede descifrar y leer el mensaje completo, lo cual contradice el propósito de la confidencialidad. Además, este esquema es extremadamente ineficiente computacionalmente porque las operaciones asimétricas sobre mensajes grandes son muy lentas comparadas con operar solo sobre un hash de tamaño fijo. En conclusión, aunque este mecanismo mantiene el no repudio y la autenticación, **viola la confidencialidad** al exponer el mensaje a cualquiera con la clave pública y presenta serios problemas de **eficiencia** al no utilizar funciones hash, lo que lo hace impráctico e inseguro para implementaciones reales.

## **2. Vulnerability to Attack Vectors**

---

In the following scenario, Oscar is an adversary. Provide a detail analysis of the potential problems with each of them, the vulnerabilities that Oscar is exploiting, and the cryptographic solutions that can be used to address them.

### **Scenario 1:**

Alice sends to Bob:
- x = "Transfer $1000 to Mark".

- the corresponding digital signature $auth(x)$

Oscar intercepts the transmission and replaces the recipient, modifying the message to:
- x = "Transfer $1000 to Oscar".

Will Bob detect this alteration?

### **Scenario 2:**

Oscar observes a valid transaction message $x$ ("Transfer 1000 to Mark") and its authentic signature $auth(x)$. Oscar intercepts and retransmits this exact message and signature 100 times to Bob.

## **2. Solución**

---

### **Scenario 1:**

**Análisis del problema**:

Oscar está intentando realizar un ***ataque de manipulación de mensajes (message tampering attack)***, donde intercepta la comunicación entre Alice y Bob y modifica el contenido del mensaje para cambiar el destinatario de la transferencia de "Mark" a "Oscar".

**Vulnerabilidad que Oscar está explotando**:

Oscar está tratando de explotar una posible falta de verificación de integridad en la comunicación. Sin embargo, en este caso, la firma digital ya está implementada correctamente.

**Detección del ataque**:

Bob sí detectará la alteración. Cuando Bob reciba el mensaje modificado $x' = $ "Transfer 1000 to Oscar" junto con la firma original $auth(x)$, procederá a verificar la firma descifrándola con la clave pública de Alice para obtener $H(x)$. Luego, Bob calculará el hash del mensaje recibido $H(x')$. Como el mensaje fue modificado, $x' ≠ x$, por lo tanto $H(x') ≠ H(x)$, y la verificación de la firma fallará. Esto alertará a Bob de que el mensaje fue alterado durante la transmisión.

**Solución criptográfica**:

La solución ya está implementada mediante el uso del protocolo estándar de firma digital que combina funciones hash criptográficas con criptografía asimétrica. Este esquema garantiza que cualquier modificación al mensaje, por mínima que sea, resultará en un hash completamente diferente, haciendo que la verificación de la firma falle y detectando así la manipulación.

---

### **Scenario 2:**

**Análisis del problema**:

Oscar está realizando un ***ataque de repetición (replay attack)***, donde captura un mensaje legítimo con su firma válida y lo retransmite múltiples veces al receptor. Aunque el mensaje y la firma son auténticos, la repetición maliciosa puede causar que la misma transacción se ejecute 100 veces, resultando en una transferencia total de $100,000$ dólares en lugar de los $1000$ dólares originales.

**Vulnerabilidad que Oscar está explotando**:

Oscar está explotando la falta de mecanismos para detectar y prevenir la reutilización de mensajes válidos. El protocolo de firma digital estándar no incluye protección contra replay attacks porque cada verificación del mensaje y la firma será exitosa, ya que ambos son legítimos. El sistema no tiene forma de distinguir entre la primera transmisión y las repeticiones subsecuentes.

**Detección del ataque**:

Bob no podrá detectar este ataque usando únicamente la firma digital, ya que el mensaje y la firma son válidos en cada una de las 100 transmisiones. Sin mecanismos adicionales, Bob procesará cada mensaje como si fuera una nueva transacción legítima.

**Solución criptográfica**:

Para prevenir ***replay attacks***, se deben implementar las siguientes soluciones criptográficas:

1. **Timestamps (marcas de tiempo)**: Alice debe incluir una marca de tiempo en cada mensaje antes de firmarlo, de modo que el mensaje sea x = "Transfer $1000 to Mark | timestamp". Bob puede entonces verificar que el timestamp está dentro de una ventana de tiempo aceptable y rechazar mensajes con timestamps antiguos o duplicados.

2. **Nonces (números usados una sola vez)**: Implementar un número de secuencia único o nonce en cada mensaje. Bob mantiene un registro de todos los nonces ya procesados y rechaza cualquier mensaje con un nonce duplicado.

3. **Session tokens**: Establecer tokens de sesión únicos que expiren después de un solo uso o después de un período de tiempo determinado.

4. **Challenge-response protocols**: Bob puede enviar un desafío (challenge) aleatorio a Alice, y Alice debe incluir este valor en su respuesta firmada, asegurando que cada transacción sea única y no pueda ser replicada.

La combinación de timestamps y nonces es generalmente la solución más robusta, ya que permite verificar tanto la frescura del mensaje como prevenir duplicados, protegiendo efectivamente contra replay attacks.

## **3. Research: Transport Layer Security (TLS 1.3)**

---

Provide a detailed analysis of the TLS 1.3 protocol. Focus on the following areas:

- The architecture of TLS, and its evolution over the last few years
- Cryptographic primitives that are being used
- How the core goals of confidentiality, integrity, and authentication are being satisfied
- Explain the modern applications in which TLS is being used

## **3. Solución**

---

### **Arquitectura y evolución**

El protocolo TLS 1.3 mantiene una estructura clara: un _handshake_ para negociación y autenticación, seguido de una _record layer_ que cifra los datos de aplicación. Con esta versión se simplificó mucho el proceso permitiendo que se reduzca la latencia (1-RTT para nuevos enlaces, opción de 0-RTT para reanudaciones) y cifrando más temprano en el flujo (IETF, 2018). Comparado con la versión previa siendo la TLS 1.2, el TLS 1.3 elimina el uso de intercambio RSA estático y de suites sin secreto perfecto hacia adelante o tambien PFS. Además,solo se admiten intercambios efímeros como (EC)DHE,  se suprimen la renegociación insegura y los modos de cifrado antiguos (CBC, RC4) (Assured AB, 2018). Por último, también se trabaja para que elementos como el SNI del _ClientHello_ queden protegidos mediante extensiones como ECH (Cloudflare, 2018).

---

### **Primitivas criptográficas empleadas**

En TLS 1.3 se emplean varias primitivas clave:

- **Intercambio de claves:** (EC)DHE efímero, lo que garantiza secreto perfecto hacia adelante.  
- **Derivacion de claves:** Se usa HKDF (“extract-then-expand”) para pasar desde el secreto primario al conjunto de claves de aplicación y reanudación (BearSSL, 2024).  
- **Cifrado autenticado:** Todas las suites de cifrado son AEAD, típicamente AES-GCM y ChaCha20-Poly1305 (Cloudflare, 2018; Assured AB, 2018).  
- **Autenticación:** El servidor debe probar su identidad mediante certificado (X.509) y la operación _CertificateVerify_. El cliente puede ser autenticado, aunque no siempre es obligatorio.

---

### **¿Cómo satisface confidencialidad, integridad y autenticación (C-I-A)?**

- **Confidencialidad:** Todos los datos de aplicación viajan cifrados con AEAD y derivan de claves efímeras, garantizando que incluso si se compromete una clave futura no se puede descifrar el tráfico pasado (IETF, 2018).  
- **Integridad:** Los modos AEAD incluyen autenticación de los datos y metadatos, y el mensaje _Finished_ asegura que el transcript del handshake no ha sido manipulado (IETF, 2018).  
- **Autenticación:** El servidor (y opcionalmente el cliente) presenta un certificado y demuestra la posesión de la clave privada. La eliminación de estructuras inseguras anteriores reduce vectores de ataque históricos (EncryptionConsulting, 2024).

---

### **Aplicaciones modernas**

TLS 1.3 es el estándar dominante en:

- **Navegación web como HTTPS y APIs**, donde los navegadores y CDNs lo adoptan por defecto para tener menor latencia y mayor seguridad (Cloudflare, 2018).  
- **Infraestructura de microservicios y bases de datos en transito**, donde se exige cifrado con PFS y AEAD para conexiones internas (EncryptionConsulting, 2024).  
- **Dispositivos móviles, IoT y redes de alto rendimiento**, donde se requiere tanto eficiencia como robustez criptográfica (Assured AB, 2018).  
- **Transporte con privacidad extendida:** la extensión ECH protege metadatos del cliente como el SNI en entornos que requieren máxima confidencialidad.


## **4. Design Problem**

---

Design an architecture for a system that requires strong, verifiable, and legally enforceable non-repudiation, utilizing cryptographic principles alongside robust protocol design.

### **Scenario:**

The application is a Secure Electronic Contract Signing System (SECS). Two parties, Alice (Service Provider) and Bob (Client), must execute a critical digital contract (e.g., an intellectual property license). The system must guarantee Non-Repudiation of Origin (neither party can deny signing the contract) and Non-Repudiation of Submission/Receipt (neither party can deny receiving the final signed contract). This is vital for maintaining the trustworthiness and legal enforceability of the digital transaction.


## **4. Solución**

---

### **Objetivo general**

El Secure Electronic Contract Signing System o SECS como se menciona,  busca garantizar que dos partes Alice (Service Provider) y Bob (Client) puedan firmar electrónicamente un contrato crítico de forma segura, verificable y legalmente vinculante.
El diseño propuesto asegura dos propiedades esenciales de no repudio:

* Non-Repudiation of Origin (NRO): donde  ninguna parte puede negar haber firmado el contrato.

* Non-Repudiation of Submission/Receipt (NRS/NRR): donde  ninguna parte puede negar haber recibido el contrato firmado final.

---

### **Principios criptográficos utilizados**

El sistema integra mecanismos modernos de criptografía asimétrica, hashing y sellado de tiempo, asegurando trazabilidad y validez legal:

|     Función                   | Mecanismo criptográfico                                                                                            |
| ------------------------- | ------------------------------------------------------------------------------------------------------------------ |
| Identidad y autenticación   | Certificados X.509 con RSA-PSS o ECDSA, emitidos por una Autoridad de Certificación confiable |
| Integridad del contrato       | SHA-256/SHA-512 como función hash                                                                              |
| Firma digital             | RSA-PSS / ECDSA firmando el hash del contrato                                                              |
| Sellado de tiempo    | Timestamp Authority (TSA) bajo RFC 3161, el cual garantiza prueba de existencia temporal                        |
| Cifrado en tránsito |   TLS 1.3 con AES-GCM o ChaCha20-Poly1305                                                              |
| Evidencia verificable       | Uso de OCSP stapling y cadena de certificados completa en cada firma                                           |

---

### **Arquitectura del sistema**
La arquitectura posee tres capas principales:

1. **Application Layer**

Interfaz web o móvil donde Alice y Bob visualizan el contrato y aprueban la firma. Incluye controles de *What you see is what you sign*  o WYSIWYS para evitar manipulación visual del documento.

2. **Security & Trust Layer**

* Módulo de firma digital: Genera el hash del documento y lo firma con la clave privada del usuario.

* TSA: Añade sello de tiempo a la firma.

* Certificate Validation Module: Verifica la validez y revocación del certificado mediante OCSP.

* Logging/Audit Service: Registra cada operación ( ya se a firma, envío, recepción ) en un almacén inmutable (por ejemplo S3 con Object Lock o blockchain privada).

3. **Communication Layer**

Utiliza TLS 1.3 para garantizar cifrado y autenticacion mutua entre los nodos del sistema, evitando el famoso man in the middle  o Replay Attacks.

---

### **Flujo de protocolo propuesto**

Se propone el siguiente diagrama de frecuencia:

    participant Alice as Alice (Service Provider)
    participant SECS as SECS Server
    participant Bob as Bob (Client)
    Nota sobre Alice,Bob: Secure Electronic Contract Signing Process

    Alice->>SECS: Upload Contract M
    SECS->>Alice: Hash(M) = H(M)
    Alice->>SECS: Sign H(M) with Private Key → Sig_A
    SECS->>TSA: Request Timestamp for Sig_A
    TSA->>SECS: Timestamped Signature (Sig_A, TS_A)
    SECS->>Bob: {M, Sig_A, Cert_A, TS_A}
    Bob->>SECS: Verify Sig_A with Cert_A, sign H(M) → Sig_B
    SECS->>TSA: Timestamp Sig_B → TS_B
    SECS->>Alice & Bob: Final Package {M, Sig_A, Sig_B, Cert_A, Cert_B, TS_A, TS_B, Audit_ID}
    Nota sobre Alice,Bob: Both receive signed receipt and evidences (NRO + NRR)

---

### **Garantía de no repudio**

| Propiedad                                           | Mecanismo que la garantiza                                                                                                                                                              |
| ---------------------------------------------------             | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Non repudiation of origin o NRO**                 | La firma digital de cada parte (RSA-PSS/ECDSA) sobre el hash del contrato y el timestamp TSA sirven como prueba de autoría. Ninguna parte puede negar haber usado su clave privada. |
| **Non repudiation of submission/receipt** | El acuse firmado por el servidor SECS, junto con los sellos de tiempo y los logs inmutables, prueban que ambas partes recibieron la misma versión del contrato.             |    
| **Integridad**                                            | Verificación cruzada de hashes y almacenamiento inmutable del contrato y evidencias.                                                                                                    |
| **Autenticación**                                          | Certificados digitales emitidos por CA confiable bajo TLS 1.3 y validación OCSP.                                                                           

---

### **Evidencia verificable**

Cada operación produce un paquete de evidencia digital o también un PED exportable y verificable por terceros. Este paquete permite verificación offline, incluso sin conexión al servidor SECS garantizando así una validez legal y técnica a largo plazo.

PED = {

            Contract: M,

            Hash: H(M),

            Signatures: [Sig_A, Sig_B],

            Certificates: [Cert_A, Cert_B],

            Timestamps: [TS_A, TS_B],

            OCSP_Responses: [OCSP_A, OCSP_B],

            Audit_ID: UUID

}

---

### **Conclusiones del diseño**

El diseño propuesto para la app SECS integra criptografía asimétrica moderna, sellos de tiempo y canales seguros TLS 1.3 para garantizar que tanto el origen como la recepción del contrato puedan ser verificados, auditados y defendidos legalmente. De esta forma, vemos que se combina seguridad técnica y fuerza probatoria, cumpliendo con los requisitos de no repudio exigidos en transacciones digitales críticas.

## **Referencias**

---

- Assured AB. (2018). *T​LS 1.3 in a nutshell*. https://www.assured.se/posts/tls-1-3-nutshell  
- BearSSL. (2024). *TLS 1.3 status — cryptographic algorithms*. https://www.bearssl.org/tls13.html  
- Cloudflare. (2018). *A detailed look at RFC 8446 (a.k.a. TLS 1.3)*. Cloudflare Blog*. https://blog.cloudflare.com/rfc-8446-aka-tls-1-3/  
- EncryptionConsulting. (2024). *Understanding TLS 1.2 and TLS 1.3*. https://www.encryptionconsulting.com/tls-1-2-and-tls-1-3/  
- Internet Engineering Task Force. (2018). *The Transport Layer Security (TLS) Protocol Version 1.3 (RFC 8446)*. https://datatracker.ietf.org/doc/html/rfc8446